In [16]:
import pandas as pd
from googlesearch import search
import wordninja
import requests
from bs4 import BeautifulSoup
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import Comp_classify_def
"""
This python file stores all definitions used in Comp_Classify_final.py
"""
def transform_data (inputseries):
    X_df = pd.DataFrame(inputseries)
    X_df['Company'] = 'Nan'
    for i in range(5):
        stri = 'Link'+str(i)
        X_df[stri] = 'Nan'
        stri = 'Data'+str(i)
        X_df[stri] = 'Nan'
    return X_df
#returns X_df (Dataframe having added columns.)
def cleanme(html):
    soup = BeautifulSoup(html) # create a new bs4 object from the html data loaded
    for script in soup(["script"]):
        script.extract()
    text = soup.get_text()
    text = text.replace('\n', ' ').replace('\r', '').replace('\t',' ')
    return text.strip()
#cleans the response from website.
def get_df_with_lmtd_desc(X):
    X_df = link_obtain(X)
    X_df = lmtd_data_desc(X_df)
    return X_df
#combines transform_data, link_obtain and lmtd_data_desc functions
def get_df_with_desc(X):
    X_df = link_obtain(X)
    X_df = data_desc(X_df)
    return X_df
#combines transform_data, link_obtain and data_desc functions
def link_obtain(X_train):
    count = 0
    X_df = transform_data(X_train)
    for i,j in X_df.iterrows():
        URL = X_df['housing_flg'][i]
        query = URL
        #print(query)
        #Directly Retrieving all ifsc code data without saving link in column.
        if(len(str(query.strip())))<5:
            t =0
            str_list = list('https://www.ifsccodebank.com/search-by-IFSC-code.aspx?IFSCCode=')
            str_list.append(query)
            url = ''.join(str_list)
            response = requests.get(''.join(str_list))
            soup = BeautifulSoup(response.text,'html.parser')
            t = soup.get_text()
            r = t[t.find(query)+7:]
            r = (r[:r.find('-')-5].strip())
            X_df['Company'][i] = r
            q2 = str(X_df['Company'][i])
            if 'com' not in q2:
                t =0
                for l in search(q2,stop = 2):
                    stri = 'Link'+str(t)
                    #print(q2,l)
                    X_df[stri][i] = l
                    count = count+1
                    t = int(t)+1
                    #print(stri)
            else:
                X_df['Company'][i] = 'Nan'
            continue
        #if Name has no space ---> Using wordninja algo to split the words.
        elif ' ' not in query.strip():
            query = " ".join(str(x) for x in wordninja.split(query))
    ####===> The following gets the link and not the data <===####
        # Direct search for appropriate official Link
        try:
            t = 0
            for k in search(query,stop = 2):
                stri = 'Link' + str(t)
                X_df[stri][i] = k
                t = t+1
                #print(stri)
        # Search the words with key word Financial Services
            t = 2
            query = URL + ' Finanancial Services'
            for k in search(query,stop = 2):
                stri = 'Link' + str(t)
                X_df[stri][i] = k
                t = t+1
                #print(stri)
            count = count+1
        except:
            # except block to store the company which has bad request/other issue
            X_df['Link4'][i] = query
            print('Errored out: ',query)
        print('Completed : ')
        print(count//15,end = '\r')
    return X_df
#takes series and returns dataframe with link obtained from the respective companies.
def lmtd_data_desc(X_df):
    for i,j in X_df.iterrows():
        if 'Nan' not in X_df['Link0'][i]:
            response = requests.get(X_df['Link0'][i],verify = False)
            t = cleanme(response.text)
            X_df['Data0'][i] = t
    return X_df
#returns dataframe with data description obtained from first link
def data_desc(X_df):
    for i,j in X_df.iterrows():
        try:
            if 'Nan' not in X_df['Link3'][i]:
                response = requests.get(X_df['Link3'][i],verify = False)
                t = cleanme(response.text)
                X_df['Data3'][i] = t
            if 'Nan' not in X_df['Link0'][i]:
                response = requests.get(X_df['Link0'][i],verify = False)
                t = cleanme(response.text)
                X_df['Data0'][i] = t
        except:
            print('Error: ',i)
    return X_df
#returns dataframe with data description obtained from first and third link.
def transform_to_vector(vectorizer,X_df):
    X_res = pd.DataFrame(columns = vectorizer.get_feature_names())
    for i,j in X_df.iterrows():
        print(X_df['housing_flg'][i])
        rtlst = [0]*len(vectorizer.get_feature_names())
        rt = vectorizer.transform([X_df['Data0'][i]]).toarray()
        for k in range(rt.shape[1]):
            rtlst[k] = rt[0][k]+rtlst[k]
            print ('Initial ',i, rtlst[k])
        rt = vectorizer.transform([X_df['Data3'][i]]).toarray()
        for k in range(rt.shape[1]):
            rtlst[k] = rt[0][k]+rtlst[k]
            print('Final ',i,rtlst[k])
        X_res.loc[i] = rtlst
    return X_res
#vectorizes the dataframe text into numbers => returns dataframe with vectorized
#text according to the features.


def process_data(filename):
    """
    Processing DataSet
    """
    #Getting Data from testdata_Labeled.csv file.
    data = pd.read_csv(filename)
    #Retrieving only data which are labeled for now
    data_df = data.head(5)
    #Dropping columns with missing value
    data_df.dropna(how='all',axis=1,inplace=True)
    data_df.dropna(how='all',axis=0,inplace=True)
    #data_df.drop(axis=1,labels=[data_df.columns[3],data_df.columns[4]],inplace=True)
    X = data_df.housing_flg
    y = data_df.Category
    return X,y
    # """
    # Splitting data into train and test sets among the category before applying
    # vectorization
    # """
    # #X_train,X_test,Y_train,Y_test = train_test_split(X,y,random_state = 1)
    #
def trans_data_to_vect(X):
    """
    get_df_with_desc transforms the series into a dataframe having
    columns filled with respective values
    """
    X_df = get_df_with_lmtd_desc(X)

    """
    Vectorizing data with TfidfVectorizer
    """
    #configuring TFidf Vectorizer with predefined features.
    vectorizer = TfidfVectorizer(use_idf = 'False',binary=True,ngram_range=(1,2))
    vectorizer.fit(['Finance', 'Loan', 'Insurance',
    'Investor' ,'Business Services','Financial Services', 'Personal Loan', 'Mobile Loan',
    'Consumer Durable Loan', 'Furniture Loan',
    '2-wheeler Loan', 'E - Bike Loan', 'Bank', 'Home Loan', 'Housing Finance'])
    vectorizer.get_feature_names()
    X_res = transform_to_vector(vectorizer,X_df)
    #X_res is a dataframe that stores the vectorized data
    return X_res
def train_test(data_df,y,num,comp_name):
    """
    Splitting data into train and test sets among the category before applying
    vectorization
    """
    X_df = data_df.head(num)

    X_train,X_test,Y_train,Y_test = train_test_split(X_df,y,random_state = 1)

    #Trying MultinomialNB and KNN classifier algos:

    categories = ['Finance','Assmt','Investor','Loan','Bank','Home Loan','other']
    print('Classification by MultinomialNB: ')
    alp = 1
    max_scr = 0
    for k in range(1):
        model = MultinomialNB(alpha = k)
        model.fit(X_train,Y_train)
        labels = model.predict(X_test)
        scr = accuracy_score(labels,Y_test)
        if scr >max_scr:
            max_scr = scr
            alpha = k
        print('score for alpha ', k,' is ' , scr)
    n = 1
    max_scr = 0
    print('Classification by K nearest Neighbor: ')
    for k in range(1):
        modelknn = KNeighborsClassifier(n_neighbors=(k+1))
        modelknn.fit(X_train,Y_train)
        labelknn = modelknn.predict(X_test)
        scr = accuracy_score(Y_test,labelknn)
        print('Accuracy for ',k+1,' neighbors = ',scr)
        if scr>max_scr:
            max_scr = scr
            n = k
    comp_name_x = pd.DataFrame([comp_name],columns = {'housing_flg'})
    comp_df = trans_data_to_vect(comp_name_x)
    model = MultinomialNB(alpha =1)
    model.fit(X_train,Y_train)
    labels = model.predict(comp_df)
    print('Multinomial NB  label: ',labels)
    modelknn = KNeighborsClassifier(n_neighbors=1)
    modelknn.fit(X_train,Y_train)
    labels = modelknn.predict(comp_df)
    print('Multinomial NB  label: ',labels)
if __name__ == '__main__':
    filename = input('Enter file with labeled data or press 2 to skip train: ')
    if filename!='2':
        X,y = process_data(filename)
        data_processed_df = trans_data_to_vect(X)
        num = input('Enter the number of data already labeled: ')
        comp_name = input('Enter company name to classify: ')
        train_test(data_processed_df,y,74,comp_name)
    else:
        filename = 'testdata_Labeled.csv'
        X,y = process_data(filename)
        data_processed_df = trans_data_to_vect(X)
        comp_name = input('Enter company name to classify: ')
        num = int(input('Enter the number of data already labeled: '))
        train_test(data_processed_df,y,num,comp_name)


Enter file with labeled data or press 2 to skip train: 2


/home/roshan/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/roshan/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Completed : 
Completed : 
Completed : 
Completed : 


/home/roshan/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/roshan/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/roshan/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/roshan/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:851: InsecureRequestWar

AADHAR HOUSING FINAN
Initial  1 0.2914856323350768
Initial  1 0.0
Initial  1 0.0
Initial  1 0.2914856323350768
Initial  1 0.0
Initial  1 0.0
Initial  1 0.0
Initial  1 0.0
Initial  1 0.0
Initial  1 0.25310618858624456
Initial  1 0.2914856323350768
Initial  1 0.2914856323350768
Initial  1 0.0
Initial  1 0.0
Initial  1 0.2914856323350768
Initial  1 0.2914856323350768
Initial  1 0.2914856323350768
Initial  1 0.2914856323350768
Initial  1 0.2914856323350768
Initial  1 0.2914856323350768
Initial  1 0.1491166523929955
Initial  1 0.0
Initial  1 0.0
Initial  1 0.0
Initial  1 0.0
Initial  1 0.25310618858624456
Initial  1 0.0
Initial  1 0.0
Final  1 0.2914856323350768
Final  1 0.0
Final  1 0.0
Final  1 0.2914856323350768
Final  1 0.0
Final  1 0.0
Final  1 0.0
Final  1 0.0
Final  1 0.0
Final  1 0.25310618858624456
Final  1 0.2914856323350768
Final  1 0.2914856323350768
Final  1 0.0
Final  1 0.0
Final  1 0.2914856323350768
Final  1 0.2914856323350768
Final  1 0.2914856323350768
Final  1 0.291485632

/home/roshan/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/roshan/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Baroda Bank
Initial  0 0.25181881262445976
Initial  0 0.0
Initial  0 0.0
Initial  0 0.25181881262445976
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.0
Initial  0 0.21866223514036554
Initial  0 0.25181881262445976
Initial  0 0.25181881262445976
Initial  0 0.0
Initial  0 0.0
Initial  0 0.25181881262445976
Initial  0 0.25181881262445976
Initial  0 0.25181881262445976
Initial  0 0.25181881262445976
Initial  0 0.25181881262445976
Initial  0 0.25181881262445976
Initial  0 0.1288241140646427
Initial  0 0.25181881262445976
Initial  0 0.0
Initial  0 0.25181881262445976
Initial  0 0.25181881262445976
Initial  0 0.21866223514036554
Initial  0 0.25181881262445976
Initial  0 0.0
Final  0 0.25181881262445976
Final  0 0.0
Final  0 0.0
Final  0 0.25181881262445976
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.0
Final  0 0.21866223514036554
Final  0 0.25181881262445976
Final  0 0.25181881262445976
Final  0 0.0
Final  0 0.0
Final  0 0.25181881262445976
Final 